In [1]:
import numpy as np
import pandas as pd
from numpy import array
from numpy import asarray
from numpy import zeros
# import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.metrics import f1_score

In [2]:
Data = pd.read_csv("Data.csv")

In [3]:
def function(question_body,question_title, tags):
  import re
  from keras.models import load_model
  import tensorflow
  import keras
  import pickle as pkl
  
  stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])
  from bs4 import BeautifulSoup
  from tqdm import tqdm
  preprocessed_questions_body = []
  preprocessed_questions_title = []
  preprocessed_tags = []

  def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = ' '.join(e.lower() for e in phrase.split() if e.lower() not in stopwords)
    # print(type(phrase))
    # print(phrase)
    return phrase
  preprocessed_questions_body = decontracted(question_body)
  preprocessed_questions_title= decontracted(question_title)
  preprocessed_tags = decontracted(tags)


  Complete = preprocessed_questions_body + " " + preprocessed_questions_title + " " + preprocessed_tags
  # print(Complete)
  with open('tokenizer.pkl', 'rb') as handle:
    tokenizeer=pkl.load(handle)
    sequences_test = tokenizeer.texts_to_sequences([Complete])
    
    padded_text_test = pad_sequences(sequences_test, maxlen=200, padding='post')
    print(padded_text_test.shape)

  model = load_model('final_model.hdf5')
  # f_names = model.feature_name
  # model.feature_names = list(fg.industry.values)
  # f_names = model.feature_names
  # print(f_names)
  predicted_industry=model.predict(padded_text_test)
  # print(predicted_industry)

  with open('vectorizer.pkl', 'rb') as handle:
    vectorizer=pkl.load(handle)
  feature_names = vectorizer.get_feature_names_out()
  feature_names = pd.DataFrame(feature_names)

  thresh = 0.01 #0.01
  for dat in predicted_industry:
    whr = np.where(dat > thresh)

    # print(whr)
  

  predicted_class = feature_names.loc[whr]
  print("Predicted classes are :", predicted_class.values)


  return predicted_class 
 


In [4]:
industry = function(question_body="i am student",question_title="i am good in computer", tags="eucational")

(1, 200)
1/1 [==============================] - 0s 186ms/step
Predicted classes are : [['computer_games']
 ['computer_networking']
 ['consumer_services']
 ['education_management']
 ['education_technology']
 ['finance_mortgage_underwriting']
 ['financial_services']
 ['financial_services_sales_technology']
 ['food_beverages']
 ['information_technology_services']
 ['law_practice']
 ['media_production']
 ['software']
 ['staffing_recruiting']
 ['technology']
 ['telecommunications']]
